In [1]:
import warnings
warnings.filterwarnings('ignore')
from pydub import AudioSegment
import pandas as pd
import os
from scipy.fftpack import fft

In [2]:
def getFiles(directory,typ='.mp3'):
    for file in os.listdir(directory):
        if file.endswith(typ):
            yield('{}/{}'.format(directory,file))
        else:
            continue

In [3]:
def splitFile(audio,len_miliseconds=5000):
    chunks = {}
    for i in range(len(audio)//len_miliseconds):
        chunks[i] = audio[i*len_miliseconds:(i+1)*len_miliseconds].get_array_of_samples()
    return chunks

In [4]:
def getFFT(dictionary,n=500):
    chunks = {}
    for i, data in dictionary.items():
        chunks[i] = pd.Series(fft(data,n))
    return pd.DataFrame(chunks).T

In [5]:
db = pd.DataFrame()
for lang in os.listdir('../data/'):
    for file in getFiles('../data/{}'.format(lang)):
        audio = AudioSegment.from_mp3(file)
        chunks = splitFile(audio)
        chunks_fft = getFFT(chunks,n=1000)
        chunks_fft['lang'] = lang
        db = pd.concat([db,chunks_fft])
db.index.name = 'Id'

In [6]:
db.to_csv('db.csv')